In [14]:
import json
import yfinance as yf
import plotly.graph_objects as go
import pandas as pd
import requests
from datetime import datetime
import os
from datetime import datetime, timedelta

energy_sector = [
    {"sector": "Oil&Gas",
     "companies": [
         {"name": "Repsol", "country": "Spain", "symbol": "REP.MC" , "data": []},
         {"name": "Exxon Mobil Corporation", "country": "USA", "symbol": "XOM", "data": []}
     ]},
    {"sector": "ElectricPower",
     "companies": [
         {"name": "Iberdrola", "country": "Spain", "symbol": "IBE.MC", "data": []},
         {"name": "NextEra Energy, Inc.", "country": "USA", "symbol": "NEE", "data": []}
     ]},
    {"sector": "RenewableEnergy",
     "companies": [
         {"name": "Acciona", "country": "Spain", "symbol": "ANA.MC", "data": []},
         {"name": "Solaria", "country": "Spain", "symbol": "SOL", "data": []},
         {"name": "Solaredge Technologies Inc.", "country": "USA", "symbol": "SEDG", "data": []}
     ]},
    {"sector": "EnergyServices",
     "companies": [
         {"name": "Siemens Gamesa Renewable Energy", "country": "Spain", "symbol": "SGRE.MC", "data": []},
         {"name": "Honeywell International Inc.", "country": "USA", "symbol": "HON", "data": []}
     ]},
    {"sector": "Infrastructure",
     "companies": [
         {"name": "Enagas", "country": "Spain", "symbol": "ENG.MC", "data": []},
         {"name": "Kinder Morgan, Inc.", "country": "USA", "symbol": "KMI", "data": []}
     ]}
]


now_date = datetime.now()


base_url = "https://ucdpapi.pcr.uu.se/api/gedevents/23.1"

start_date = datetime(2001, 1, 1)  
end_date = datetime(now_date.year , now_date.month , now_date.day )

all_data = []

current_date = start_date
parameters = {
    "pagesize": 10000, 
    "format": "json",        
    "date_start": start_date.strftime('%Y-%m-%d'),  
    "date_end": end_date.strftime('%Y-%m-%d') 
}

response = requests.get(base_url, params=parameters)
data = json.loads(response.text)
page = 1
total_pages = data['TotalPages']

print(total_pages)
print(page)
next_page = data['NextPageUrl']
if response.status_code == 200:
    all_data.extend(data['Result'])

while page <=total_pages:    
    try:
        if(next_page!=""):
            response = requests.get(next_page)    
            data = json.loads(response.text)
            next_page = data['NextPageUrl']
            if response.status_code == 200:      
                
                all_data.extend(data['Result'])
                df = pd.DataFrame(all_data)
        
            else:
                print(f"Error al obtener los datos del UCDP para la fecha {current_date}.")
            print(total_pages)
            print(page)
            page=page+1
        else:
            break
        
    except:
        print("Error")
    
    

# Guardar los datos en un archivo CSV
df.to_csv('ucdp_data_events_2000_2024.csv', index=False)
base_url = "https://ucdpapi.pcr.uu.se/api/ucdpprioconflict/23.1"

start_date = datetime(2001, 1, 1)  
end_date = datetime(now_date.year , now_date.month , now_date.day )

all_data = []

current_date = start_date
parameters = {
    "pagesize": 10000, 
    "format": "json",        
    "date_start": start_date.strftime('%Y-%m-%d'),  
    "date_end": end_date.strftime('%Y-%m-%d') 
}

response = requests.get(base_url, params=parameters)
data = json.loads(response.text)
page = 1
total_pages = data['TotalPages']

print(total_pages)
print(page)
next_page = data['NextPageUrl']
if response.status_code == 200:
    all_data.extend(data['Result'])

while page < total_pages:    
    response = requests.get(next_page)    
    data = json.loads(response.text)
    next_page = data['NextPageUrl']
    if response.status_code == 200:      
        
        all_data.extend(data['Result'])
        df = pd.DataFrame(all_data)

        # Guardar los datos en un archivo CSV
    else:
        print(f"Error al obtener los datos del UCDP para la fecha {current_date}.")
    print(total_pages)    
    page=page+1
    print(page)

df.to_csv('ucdp_data_conflicts_2000_2024.csv', index=False)   

if not os.path.exists('docs'):
    os.makedirs('docs')

for sector in energy_sector:
    for company in sector['companies']:
        company_data = yf.download(company['symbol'], start='2000-01-01', end=now_date)        
        file_name = f"{company['name']}_data.csv"
        company_data.to_csv(os.path.join('docs', file_name))  
        company['data'] = company_data

32
1
32
1
32
2
32
3
32
4
32
5
32
6
32
7
32
8
32
9
32
10
32
11
32
12
32
13
32
14
32
15
32
16
32
17
32
18
32
19
32
20
32
21
32
22
32
23
32
24
32
25
32
26
32
27
32
28
32
29
32
30
32
31
1
1


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['SGRE.MC']: Exception('%ticker%: No timezone found, symbol may be delisted')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [24]:
# Cargar los datos de conflictos y eventos desde los CSV
df_conflictos = pd.read_csv('ucdp_data_conflicts_2000_2024.csv', low_memory=False)

df_eventos = pd.read_csv('ucdp_data_events_2000_2024.csv', low_memory=False)

df_resultado_union = pd.merge(df_conflictos, df_eventos, on='id', how='inner')
df_resultado_union_div = df_resultado_union.groupby(by='year_x')

# Guardar los conflictos con eventos del año 2010 en un CSV
df_resultado_union_div.get_group(2010).to_csv('docs/conflicts_with_events_2010.csv', index=False)


In [19]:

df_resultado_union.head()

,id,relid_x,year_x,active_year_x,code_status_x,type_of_violence_x,conflict_dset_id_x,conflict_new_id_x,conflict_name_x,dyad_dset_id_x,...,date_end_y,deaths_a_y,deaths_b_y,deaths_civilians_y,deaths_unknown_y,best_y,high_y,low_y,gwnoa_y,gwnob_y
0,4,YEM-2010-1-784-46,2010,True,Clear,1,230,230,Yemen (North Yemen): Government,459,...,2010-09-25T00:00:00,2,0,0,0,2,2,2,678,NaN
1,5,YEM-2011-3-678-28,2011,True,Clear,3,123,715,Government of Yemen (North Yemen) - Civilians,123,...,2011-02-19T00:00:00,0,0,0,0,0,2,0,678,NaN
2,6,YEM-2011-1-784-30,2011,True,Clear,1,230,230,Yemen (North Yemen): Government,459,...,2011-04-16T00:00:00,0,0,0,0,0,1,0,678,NaN
3,7,YEM-2012-1-784-161,2012,True,Clear,1,230,230,Yemen (North Yemen): Government,459,...,2012-06-04T00:00:00,4,1,0,0,5,5,5,678,NaN
4,10,YEM-2012-1-784-254,2012,True,Clear,1,230,230,Yemen (North Yemen): Government,459,...,2012-12-09T00:00:00,0,4,1,0,5,5,5,678,NaN


In [14]:
import requests

base_url = "https://api.gdeltproject.org/api/v2/"
endpoint = "events"

params = {
    "format": "json",  # Formato de los datos de respuesta
    "query": "conflict OR terrorism OR territorial dispute",  # Consulta de búsqueda
    # Otros parámetros según la documentación de la API de GDELT
}

# URL completa para la consulta
api_url = base_url + endpoint

# Realizar la llamada a la API
response = requests.get(api_url, params=params)

# Verificar el estado de la respuesta
if response.status_code == 200:
    # Procesar los datos de respuesta (en formato JSON)
    data = response.json()
    # Hacer algo con los datos...
else:
    # Manejar errores de la llamada a la API
    print("Error al llamar a la API:", response.status_code)

KeyboardInterrupt: 

In [2]:
import pandas as pd

In [3]:
wars_in_world_data = pd.read_csv('ucdp_data_2000_2024.csv')
wars_in_world_data

,id,relid,year,active_year,code_status,type_of_violence,conflict_dset_id,conflict_new_id,conflict_name,dyad_dset_id,...,date_end,deaths_a,deaths_b,deaths_civilians,deaths_unknown,best,high,low,gwnoa,gwnob
0,4,YEM-2010-1-784-46,2010,True,Clear,1,230,230,Yemen (North Yemen): Government,459,...,2010-09-25T00:00:00,2,0,0,0,2,2,2,678.0,NaN
1,5,YEM-2011-3-678-28,2011,True,Clear,3,123,715,Government of Yemen (North Yemen) - Civilians,123,...,2011-02-19T00:00:00,0,0,0,0,0,2,0,678.0,NaN
2,6,YEM-2011-1-784-30,2011,True,Clear,1,230,230,Yemen (North Yemen): Government,459,...,2011-04-16T00:00:00,0,0,0,0,0,1,0,678.0,NaN
3,7,YEM-2012-1-784-161,2012,True,Clear,1,230,230,Yemen (North Yemen): Government,459,...,2012-06-04T00:00:00,4,1,0,0,5,5,5,678.0,NaN
4,10,YEM-2012-1-784-254,2012,True,Clear,1,230,230,Yemen (North Yemen): Government,459,...,2012-12-09T00:00:00,0,4,1,0,5,5,5,678.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239995,15954,NIG-2009-1-793-10,2009,True,Clear,1,297,297,Nigeria: Government,640,...,2009-07-30T00:00:00,0,0,0,0,0,1,0,475.0,NaN
239996,15955,NIG-2010-3-1935-4,2010,True,Clear,3,1051,1850,Jama'atu Ahlis Sunna Lidda'awati wal-Jihad - C...,1051,...,2010-10-06T00:00:00,0,0,1,0,1,1,0,NaN,NaN
239997,15956,SAF-1993-2-312-34,1993,True,Clear,2,5450,4840,Supporters of ANC - Supporters of IFP,5450,...,1993-12-07T00:00:00,1,0,1,0,2,2,2,NaN,NaN
239998,15957,NIG-2010-1-793-10,2010,False,Clear,1,297,297,Nigeria: Government,640,...,2010-11-13T00:00:00,0,0,0,0,0,1,0,475.0,NaN


In [4]:
wars_in_world_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 49 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 240000 non-null  int64  
 1   relid              240000 non-null  object 
 2   year               240000 non-null  int64  
 3   active_year        240000 non-null  bool   
 4   code_status        240000 non-null  object 
 5   type_of_violence   240000 non-null  int64  
 6   conflict_dset_id   240000 non-null  int64  
 7   conflict_new_id    240000 non-null  int64  
 8   conflict_name      240000 non-null  object 
 9   dyad_dset_id       240000 non-null  int64  
 10  dyad_new_id        240000 non-null  int64  
 11  dyad_name          240000 non-null  object 
 12  side_a_dset_id     240000 non-null  int64  
 13  side_a_new_id      240000 non-null  int64  
 14  side_a             240000 non-null  object 
 15  side_b_dset_id     240000 non-null  int64  
 16  si

In [5]:
wars_in_world_data.head()

,id,relid,year,active_year,code_status,type_of_violence,conflict_dset_id,conflict_new_id,conflict_name,dyad_dset_id,...,date_end,deaths_a,deaths_b,deaths_civilians,deaths_unknown,best,high,low,gwnoa,gwnob
0,4,YEM-2010-1-784-46,2010,True,Clear,1,230,230,Yemen (North Yemen): Government,459,...,2010-09-25T00:00:00,2,0,0,0,2,2,2,678.0,NaN
1,5,YEM-2011-3-678-28,2011,True,Clear,3,123,715,Government of Yemen (North Yemen) - Civilians,123,...,2011-02-19T00:00:00,0,0,0,0,0,2,0,678.0,NaN
2,6,YEM-2011-1-784-30,2011,True,Clear,1,230,230,Yemen (North Yemen): Government,459,...,2011-04-16T00:00:00,0,0,0,0,0,1,0,678.0,NaN
3,7,YEM-2012-1-784-161,2012,True,Clear,1,230,230,Yemen (North Yemen): Government,459,...,2012-06-04T00:00:00,4,1,0,0,5,5,5,678.0,NaN
4,10,YEM-2012-1-784-254,2012,True,Clear,1,230,230,Yemen (North Yemen): Government,459,...,2012-12-09T00:00:00,0,4,1,0,5,5,5,678.0,NaN


In [13]:
grouped_by_type_of_violence = wars_in_world_data[wars_in_world_data['type_of_violence']>1]
grouped_by_type_of_violence.head()

,id,relid,year,active_year,code_status,type_of_violence,conflict_dset_id,conflict_new_id,conflict_name,dyad_dset_id,...,date_end,deaths_a,deaths_b,deaths_civilians,deaths_unknown,best,high,low,gwnoa,gwnob
1,5,YEM-2011-3-678-28,2011,True,Clear,3,123,715,Government of Yemen (North Yemen) - Civilians,123,...,2011-02-19T00:00:00,0,0,0,0,0,2,0,678.0,NaN
66,100,YEM-2011-3-678-2,2011,True,Clear,3,123,715,Government of Yemen (North Yemen) - Civilians,123,...,2011-02-25T00:00:00,0,0,4,0,4,4,4,678.0,NaN
67,101,YEM-2011-3-678-3,2011,True,Clear,3,123,715,Government of Yemen (North Yemen) - Civilians,123,...,2011-02-16T00:00:00,0,0,1,0,1,1,1,678.0,NaN
68,102,YEM-2011-3-678-4,2011,True,Clear,3,123,715,Government of Yemen (North Yemen) - Civilians,123,...,2011-02-18T00:00:00,0,0,4,0,4,4,4,678.0,NaN
69,103,YEM-2011-3-678-5,2011,True,Clear,3,123,715,Government of Yemen (North Yemen) - Civilians,123,...,2011-02-20T00:00:00,0,0,1,0,1,1,1,678.0,NaN
